# Init

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import fsolve
from scipy.misc import derivative

import warnings
warnings.filterwarnings('ignore')



In [2]:
data = {
    'Price': [0.1, 0.3, 0.45, 0.7, 0.8, 1.05, 1.2, 1.25, 1.31, 1.4, 1.47, 1.55],
    'Demand': [100, 69, 58, 40, 35, 20, 18, 17, 19, 21, 18, 15],
    'Supply': [10, 25, 39, 52, 60, 84, 91, 95, 97, 100, 105, 108]
}

df = pd.DataFrame(data)

# Main

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

axs[0].plot(df['Price'], df['Demand'], marker='o', linestyle='-', color='blue')
axs[0].set_title('Price vs Demand')
axs[0].set_xlabel('Price')
axs[0].set_ylabel('Demand')


axs[1].plot(df['Price'], df['Supply'], marker='o', linestyle='-', color='red')
axs[1].set_title('Price vs Supply')
axs[1].set_xlabel('Price')
axs[1].set_ylabel('Supply')

plt.tight_layout()
plt.show()

In [15]:
def fit_polynomial_model(x, y, max_degree=5):
    scores = []
    models = []
    for degree in range(1, max_degree + 1):
        poly_features = PolynomialFeatures(degree=degree)
        x_poly = poly_features.fit_transform(x.reshape(-1, 1))

        model = LinearRegression()
        model.fit(x_poly, y)
        y_pred = model.predict(x_poly)

        score = r2_score(y, y_pred)
        scores.append(score)
        models.append((model, poly_features))

    best_score = max(scores)
    best_index = scores.index(best_score)
    best_degree = best_index + 1
    best_model, best_poly_features = models[best_index]

    return best_degree, best_score, best_model, best_poly_features

# Підбір моделей для попиту та пропозиції
price = df['Price'].values
demand = df['Demand'].values
supply = df['Supply'].values

demand_degree, demand_score, demand_model, demand_poly_features = fit_polynomial_model(price, demand)
supply_degree, supply_score, supply_model, supply_poly_features = fit_polynomial_model(price, supply)


def polynomial_model_to_string(coef, intercept):
    terms = [f"{intercept:.2f}"] + [f"{coef[i]:.2f}x^{i}" for i in range(1, len(coef))]
    return " + ".join(terms).replace("+ -", "- ")

demand_model_str = polynomial_model_to_string(demand_model.coef_, demand_model.intercept_)
supply_model_str = polynomial_model_to_string(supply_model.coef_, supply_model.intercept_)

print(f"Demand model: y = {demand_model_str}")
print(f"Supply model: y = {supply_model_str}")


Demand model: y = 133.42 - 428.26x^1 + 1085.71x^2 - 1525.61x^3 + 992.85x^4 - 236.15x^5
Supply model: y = -6.99 + 210.00x^1 - 506.89x^2 + 755.31x^3 - 483.43x^4 + 110.11x^5


In [ ]:
def draw_qp_chart(eq = False,):
    prices = np.linspace(min(data['Price']), max(data['Price']), 500)

    demand_values = demand_poly_features.transform(prices.reshape(-1, 1))
    supply_values = supply_poly_features.transform(prices.reshape(-1, 1))
    demand_plot = demand_model.predict(demand_values)
    supply_plot = supply_model.predict(supply_values)


    plt.figure(figsize=(10, 6))
    plt.plot(prices, demand_plot, label='Demand Curve', color='blue')
    plt.plot(prices, supply_plot, label='Supply Curve', color='green')
    if eq != False:
        plt.scatter(eq[0],eq[1], color='red', alpha=1, label='Equilibrium')
        plt.text(eq[0],eq[1], f'Equilibrium\n({equilibrium_price:.2f}, {equilibrium_demand:.2f})', verticalalignment='bottom', color ='red')
    plt.scatter(data['Price'], data['Demand'], color='blue', alpha=0.5, label='Actual Demand')
    plt.scatter(data['Price'], data['Supply'], color='green', alpha=0.5, label='Actual Supply')
    plt.title('Demand and Supply Curves')
    plt.xlabel('Price')
    plt.ylabel('Quantity')
    plt.legend()
    plt.grid(True)
    plt.show()
draw_qp_chart()



In [79]:
# Функція для обчислення різниці між попитом і пропозицією
def demand_supply_difference(price):
    demand_poly = demand_poly_features.transform([price])
    supply_poly = supply_poly_features.transform([price])
    demand_val = demand_model.predict(demand_poly)
    supply_val = supply_model.predict(supply_poly)
    return demand_val - supply_val


initial_guess = 1.0
equilibrium_price = fsolve(demand_supply_difference, initial_guess)[0]
equilibrium_demand = demand_model.predict(demand_poly_features.transform([[equilibrium_price]]))[0]
equilibrium = (equilibrium_price, equilibrium_demand)
print(f"Equilibrium Price: {equilibrium_price:.2f}")
print(f"Equilibrium Demand/Supply: {equilibrium_demand:.2f}")


Equilibrium Price: 0.62
Equilibrium Demand/Supply: 46.87


In [ ]:
draw_qp_chart(eq=equilibrium)

In [81]:
def demand_function(price):
    return demand_model.predict(demand_poly_features.transform([[price]]))[0]
def supply_function(price):
    return supply_model.predict(supply_poly_features.transform([[price]]))[0]


demand_slope = derivative(demand_function, equilibrium_price, dx=1e-6) # Ed
supply_slope = derivative(supply_function, equilibrium_price, dx=1e-6) # Es

if abs(demand_slope) > abs(supply_slope):
    stability = "stable"
else:
    stability = "unstable"

print(f"Demand slope at equilibrium: {demand_slope:.2f}")
print(f"Supply slope at equilibrium: {supply_slope:.2f}")
print(f"The equilibrium is {stability}.")


Demand slope at equilibrium: -69.18
Supply slope at equilibrium: 72.78
The equilibrium is unstable.


In [82]:
P1, P2 = data['Price'][0], data['Price'][-1]
Q1_demand, Q2_demand = data['Demand'][0], data['Demand'][-1]
Q1_supply, Q2_supply = data['Supply'][0], data['Supply'][-1]

# Calculating arc elasticity for demand
arc_elasticity_demand = ((Q2_demand - Q1_demand) / ((Q2_demand + Q1_demand) / 2)) / ((P2 - P1) / ((P2 + P1) / 2))

# Calculating arc elasticity for supply
arc_elasticity_supply = ((Q2_supply - Q1_supply) / ((Q2_supply + Q1_supply) / 2)) / ((P2 - P1) / ((P2 + P1) / 2))

arc_elasticity_demand, arc_elasticity_supply


(-0.8410794602698651, 0.9450613676212742)

In [106]:
taxes_demand = 0.2
taxes_supply = 0

# Ensure the price is reshaped correctly within the demand and adjusted supply functions
def adjusted_demand_function(price):
    price_reshaped = np.array(price).reshape(-1, 1)
    price_before_tax = price_reshaped + taxes_demand
    return demand_model.predict(demand_poly_features.transform(price_before_tax))[0]

def adjusted_supply_function(price):
    price_reshaped = np.array(price).reshape(-1, 1)
    price_before_tax = price_reshaped - taxes_supply
    return supply_model.predict(supply_poly_features.transform(price_before_tax))[0]


new_equilibrium_price = fsolve(lambda price: adjusted_demand_function(price) - adjusted_supply_function(price), initial_guess)[0]
new_equilibrium_quantity = supply_function(new_equilibrium_price)



In [ ]:
price_range = np.linspace(min(data['Price']), max(data['Price']), 100)
original_demand_values = [demand_function(price) for price in price_range]
adjusted_demand_values = [adjusted_demand_function(price) for price in price_range]
original_supply_values = [supply_function(price) for price in price_range]
adjusted_supply_values = [adjusted_supply_function(price) for price in price_range]


plt.figure(figsize=(10, 6))
plt.plot(price_range, original_demand_values, label='Original Demand Curve', color='blue', linestyle='dashed')
plt.plot(price_range, adjusted_demand_values, label='Adjusted Demand Curve for Taxes', color='green')
plt.plot(price_range, original_supply_values, label='Original Supply Curve', color='red')
# plt.plot(price_range, adjusted_supply_values, label='Adjusted Supply Curve for Tax', color='green')

# Mark original and new equilibrium
plt.scatter(equilibrium_price, equilibrium_demand, color='red', zorder=5)
plt.text(equilibrium_price, equilibrium_demand, f'Original Equilibrium\n({equilibrium_price:.2f}, {equilibrium_demand:.2f})\n', verticalalignment='bottom')

plt.scatter(new_equilibrium_price, new_equilibrium_quantity, color='orange', zorder=5)
plt.text(new_equilibrium_price, new_equilibrium_quantity, f'New Equilibrium\n({new_equilibrium_price:.2f}, {new_equilibrium_quantity:.2f})', verticalalignment='top')

plt.xlabel('Price')
plt.ylabel('Quantity')
plt.title('Market Equilibrium with 0.2$ Tax on Demand')
plt.legend()
plt.grid(True)
plt.show()
